In [1]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
session = requests.session()

In [2]:
%store -r current_players
%store -r gameweeks

In [3]:
current_players

,team_id,player_id,team_name,round_score,player_name,gameweek
0,333932,169,All the Smallings,0,Schmeichel,26
1,297336,397,Childish Gbaminos,0,Noble,26
2,333932,220,All the Smallings,0,David Silva,26
3,336116,389,Stop Crying Lampard,0,Fabianski,26
4,300296,211,Ducks of Bethlehem,0,Agüero,26
...,...,...,...,...,...,...
85,299717,505,Obi-Wan Iwobi,0,James,26
86,339264,524,Neuer Gonna Give You,0,Sarr,26
87,300296,589,Ducks of Bethlehem,0,Minamino,26
88,299717,615,Obi-Wan Iwobi,0,Bergwijn,26


In [4]:
gameweek = 26
current_players["team_id"][0]

'333932'

In [5]:
# make list of all team ids in the league
team_ids = []
for i in range(len(current_players)):
    if current_players["team_id"][i] not in team_ids:
        team_ids.append(current_players["team_id"][i])

In [6]:
team_ids

['333932', '297336', '336116', '300296', '339264', '299717']

In [44]:
league_info = session.get("https://draft.premierleague.com/api/entry/333932/event/1")

In [45]:
parsed_player_gameweek = json.loads(league_info.text)

In [46]:
# investigate json structure
parsed_player_gameweek

{'picks': [{'element': 149,
   'position': 1,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 204,
   'position': 2,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 122,
   'position': 3,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 334,
   'position': 4,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 346,
   'position': 5,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 267,
   'position': 6,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 448,
   'position': 7,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 134,
   'position': 8,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplier': 1},
  {'element': 113,
   'position': 9,
   'is_captain': False,
   'is_vice_captain': False,
   'multiplie

In [10]:
# This is one player id in this team this week
for i in parsed_player_gameweek["subs"][i]["element_in"]
if 267 in parsed_player_gameweek["subs"][0]["element_in"]:
    print("yes")

SyntaxError: invalid syntax (<ipython-input-10-bee491808d89>, line 2)

In [33]:
parsed_player_gameweek["picks"]

[{'element': 169,
  'position': 1,
  'is_captain': False,
  'is_vice_captain': False,
  'multiplier': 1},
 {'element': 204,
  'position': 2,
  'is_captain': False,
  'is_vice_captain': False,
  'multiplier': 1},
 {'element': 295,
  'position': 3,
  'is_captain': False,
  'is_vice_captain': False,
  'multiplier': 1},
 {'element': 299,
  'position': 4,
  'is_captain': False,
  'is_vice_captain': False,
  'multiplier': 1},
 {'element': 267,
  'position': 5,
  'is_captain': False,
  'is_vice_captain': False,
  'multiplier': 1},
 {'element': 417,
  'position': 6,
  'is_captain': False,
  'is_vice_captain': False,
  'multiplier': 1},
 {'element': 448,
  'position': 7,
  'is_captain': False,
  'is_vice_captain': False,
  'multiplier': 1},
 {'element': 29,
  'position': 8,
  'is_captain': False,
  'is_vice_captain': False,
  'multiplier': 1},
 {'element': 339,
  'position': 9,
  'is_captain': False,
  'is_vice_captain': False,
  'multiplier': 1},
 {'element': 167,
  'position': 10,
  'is_capta

In [38]:
list(parsed_player_gameweek["subs"]["element_in"])

TypeError: list indices must be integers or slices, not str

In [35]:
for item in parsed_player_gameweek["subs"]:
    print(item["element_in"])

295
417


In [25]:
for el in parsed_player_gameweek['picks']:
    print(type(el["position"]))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [49]:
def player_gameweek(team_id,weeks):
    # Will return the players in a team during all requested weeks. Expects as arguments a team id from fantasy draft premier league and the number of weeks you want data from.
    player_df = pd.DataFrame()


    for i in range(1,weeks+1):
        url = "https://draft.premierleague.com/api/entry/" + str(team_id)+ "/event/"+ str(i)
        print(i)
        player_gameweek = session.get(url)
        parsed_player_gameweek = json.loads(player_gameweek.text)
        players = []
        gameweek = []
        team_ids = []
        subbed_in = []
        subbed_out = []
        subbed = []
        position = []

        temp_df = pd.DataFrame()
        for item in parsed_player_gameweek["subs"]:
            subbed_in.append(item["element_in"])
            subbed_out.append(item["element_out"])
        for el in parsed_player_gameweek['picks']:
            team_ids.append(team_id)
            players.append(el["element"])
            position.append(el["position"])
            gameweek.append(i)
            if el["element"] in subbed_in:
                subbed.append("Subbed In")
            elif el["element"] in subbed_out:
                subbed.append("Subbed Out")
            else:
                subbed.append("N/A")

            
                    
            

        temp_df["team_id"] = team_ids
        temp_df["gameweek"] = gameweek 
        temp_df["player_id"] = players
        temp_df["position"] = position
        temp_df["subbed"] = subbed

        player_df = player_df.append(temp_df)

    return player_df



    
    



In [50]:
player_gameweek(333932,25)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


,team_id,gameweek,player_id,position,subbed
0,333932,1,149,1,N/A
1,333932,1,204,2,N/A
2,333932,1,122,3,Subbed In
3,333932,1,334,4,N/A
4,333932,1,346,5,N/A
...,...,...,...,...,...
10,333932,25,12,11,N/A
11,333932,25,149,12,N/A
12,333932,25,220,13,Subbed Out
13,333932,25,203,14,N/A


In [52]:
all_players_df = pd.DataFrame()
for team in team_ids:
    all_players_df = all_players_df.append(player_gameweek(team,25))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [53]:
all_players_df

,team_id,gameweek,player_id,position,subbed
0,333932,1,149,1,N/A
1,333932,1,204,2,N/A
2,333932,1,122,3,Subbed In
3,333932,1,334,4,N/A
4,333932,1,346,5,N/A
...,...,...,...,...,...
10,299717,25,363,11,N/A
11,299717,25,14,12,N/A
12,299717,25,522,13,N/A
13,299717,25,505,14,N/A


In [17]:
# There are API endpoints with player scores for each week but each week is a different url. I will start by checking the layout of these pages

In [19]:
# 6 teams in the league, 15 players per week, 25 weeks
6*15*25

2250

In [58]:

player_scores_gameweek1 = session.get("https://fantasy.premierleague.com/api/event/1/live")

In [59]:
player_scores_gameweek1_json = json.loads(player_scores_gameweek1.text)

In [60]:
player_scores_gameweek1_json


': 8,
    'in_dreamteam': False},
   'explain': [{'fixture': 2,
     'stats': [{'identifier': 'minutes', 'points': 1, 'value': 8},
      {'identifier': 'goals_scored', 'points': 4, 'value': 1},
      {'identifier': 'bonus', 'points': 3, 'value': 3}]}]},
  {'id': 303,
   'stats': {'minutes': 62,
    'goals_scored': 0,
    'assists': 0,
    'clean_sheets': 0,
    'goals_conceded': 1,
    'own_goals': 0,
    'penalties_saved': 0,
    'penalties_missed': 0,
    'yellow_cards': 0,
    'red_cards': 0,
    'saves': 0,
    'bonus': 0,
    'bps': 0,
    'influence': '2.0',
    'creativity': '5.3',
    'threat': '8.0',
    'ict_index': '1.5',
    'total_points': 2,
    'in_dreamteam': False},
   'explain': [{'fixture': 2,
     'stats': [{'identifier': 'minutes', 'points': 2, 'value': 62}]}]},
  {'id': 312,
   'stats': {'minutes': 0,
    'goals_scored': 0,
    'assists': 0,
    'clean_sheets': 0,
    'goals_conceded': 0,
    'own_goals': 0,
    'penalties_saved': 0,
    'penalties_missed': 0,
   

In [103]:

for item in player_scores_gameweek1_json['elements']:
    if item['id'] == 346:
        print(item)

{'id': 346, 'stats': {'minutes': 87, 'goals_scored': 0, 'assists': 0, 'clean_sheets': 0, 'goals_conceded': 1, 'own_goals': 0, 'penalties_saved': 0, 'penalties_missed': 0, 'yellow_cards': 1, 'red_cards': 0, 'saves': 0, 'bonus': 0, 'bps': 15, 'influence': '11.6', 'creativity': '38.3', 'threat': '39.0', 'ict_index': '8.9', 'total_points': 1, 'in_dreamteam': False}, 'explain': [{'fixture': 6, 'stats': [{'identifier': 'minutes', 'points': 2, 'value': 87}, {'identifier': 'yellow_cards', 'points': -1, 'value': 1}]}]}


In [62]:
# I'll create a dictionary with the contents of each page so I don't end up sending a couple of thousand requests to their server

all_weeks_dict = {}
for i in range(1,26):
    #remember I'm getting 25 weeks worth of data change this as necessary for yourself
    url = "https://fantasy.premierleague.com/api/event/"+ str(i)+"/live"
    player_scores_gameweek = session.get(url)
    all_weeks_dict[i] = json.loads(player_scores_gameweek.text)
    




In [63]:
all_weeks_dict[1]["elements"][0]

{'id': 14,
 'stats': {'minutes': 90,
  'goals_scored': 0,
  'assists': 0,
  'clean_sheets': 1,
  'goals_conceded': 0,
  'own_goals': 0,
  'penalties_saved': 0,
  'penalties_missed': 0,
  'yellow_cards': 0,
  'red_cards': 0,
  'saves': 2,
  'bonus': 1,
  'bps': 27,
  'influence': '18.0',
  'creativity': '0.0',
  'threat': '0.0',
  'ict_index': '1.8',
  'total_points': 7,
  'in_dreamteam': False},
 'explain': [{'fixture': 10,
   'stats': [{'identifier': 'minutes', 'points': 2, 'value': 90},
    {'identifier': 'clean_sheets', 'points': 4, 'value': 1},
    {'identifier': 'bonus', 'points': 1, 'value': 1}]}]}

In [64]:
all_players_df.reset_index(drop=True)

,team_id,gameweek,player_id,position,subbed
0,333932,1,149,1,N/A
1,333932,1,204,2,N/A
2,333932,1,122,3,Subbed In
3,333932,1,334,4,N/A
4,333932,1,346,5,N/A
...,...,...,...,...,...
2245,299717,25,363,11,N/A
2246,299717,25,14,12,N/A
2247,299717,25,522,13,N/A
2248,299717,25,505,14,N/A


In [65]:
id = []
total_points = []
gameweek = []
for j in all_weeks_dict:
        for x in all_weeks_dict[j]['elements']:
                print(x['stats']['total_points'])
                id.append(x["id"])
                total_points.append(x['stats']['total_points'])
                gameweek.append(j)



1
0
2
2
0
0
0
0
0
2
2
0
2
1
2
1
0
2
1
0
0
1
1
1
0
0
7
1
2
0
2
0
0
1
1
0
0
0
0
10
0
14
5
1
1
5
8
6
0
1
9
2
1
1
7
0
0
2
0
1
13
0
0
0
0
2
0
3
0
0
0
0
1
3
0
6
0
0
4
8
0
3
0
0
2
0
6
0
3
0
0
4
0
0
6
0
0
0
0
2
0
2
2
0
0
3
0
0
0
0
0
0
5
6
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
0
1
1
0
0
0
1
0
9
8
0
0
0
5
0
8
0
1
1
0
0
0
0
0
14
0
6
2
0
0
0
2
1
1
0
1
1
0
6
6
7
0
0
2
2
2
2
0
2
1
8
2
2
0
2
0
0
0
6
0
0
1
0
0
0
2
2
0
2
0
0
0
0
0
2
2
0
0
0
0
2
1
0
1
6
0
5
0
6
0
11
6
8
0
6
2
1
1
2
0
10
2
0
0
2
2
1
0
0
0
2
0
1
2
0
0
0
2
12
0
0
0
1
1
0
1
0
0
0
0
4
2
1
1
2
0
0
0
7
0
2
0
0
2
0
1
0
0
2
5
7
1
1
1
0
1
0
0
0
0
7
2
1
1
9
0
9
0
5
1
0
0
0
0
0
1
0
0
0
3
2
0
0
2
1
13
2
2
0
0
1
0
0
6
0
0
0
0
2
0
0
2
3
0
0
0
3
0
0
0
0
5
2
0
1
2
0
3
0
0
9
1
3
0
0
0
2
1
1
0
1
2
2
0
6
6
10
6
2
0
0
0
0
2
11
0
0
1
2
0
0
0
0
0
2
1
0
7
1
1
2
2
0
0
5
3
0
0
6
11
1
3
3
0
1
0
0
0
6
2
0
2
0
2
0
0
2
3
1
0
0
2
0
0
3
1
0
2
3
3
3
1
0
2
3
2
2
0
0
0
6
0
0
2
1
0
7
2
1
2
0
1
0
0
0
9
2
1
2
0
0
2
0
0
1
2
1
0
1
1
2
0
0
2
0
1
2
0
2
0
0


In [66]:
player_scores_df  = pd.DataFrame()

In [67]:
player_scores_df['player_id'] = id
player_scores_df['total_score'] = total_points
player_scores_df['gameweek'] = gameweek

In [68]:
player_scores_df

,player_id,total_score,gameweek
0,14,7,1
1,27,0,1
2,47,9,1
3,48,0,1
4,72,0,1
...,...,...,...
14106,581,0,25
14107,585,0,25
14108,592,0,25
14109,613,8,25


In [70]:
x = 0
round_score = []
for i in range(len(all_players_df)):
    x += 1
    if x % 100 == 0:
        print(x)
    for j in range(len(player_scores_df)):
        if all_players_df["player_id"].iloc[i] == player_scores_df["player_id"].iloc[j] and all_players_df["gameweek"].iloc[i] == player_scores_df["gameweek"].iloc[j]:
            round_score.append(player_scores_df["total_score"].iloc[j])


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [72]:
all_players_df['round_score'] = round_score

In [73]:
all_players_df

,team_id,gameweek,player_id,position,subbed,round_score
0,333932,1,149,1,N/A,0
1,333932,1,204,2,N/A,0
2,333932,1,122,3,Subbed In,8
3,333932,1,334,4,N/A,0
4,333932,1,346,5,N/A,1
...,...,...,...,...,...,...
10,299717,25,363,11,N/A,5
11,299717,25,14,12,N/A,7
12,299717,25,522,13,N/A,0
13,299717,25,505,14,N/A,2


In [74]:
%store all_players_df

Stored 'all_players_df' (DataFrame)


In [75]:
current_players

,team_id,player_id,team_name,round_score,player_name,gameweek
0,333932,169,All the Smallings,0,Schmeichel,26
1,297336,397,Childish Gbaminos,0,Noble,26
2,333932,220,All the Smallings,0,David Silva,26
3,336116,389,Stop Crying Lampard,0,Fabianski,26
4,300296,211,Ducks of Bethlehem,0,Agüero,26
...,...,...,...,...,...,...
85,299717,505,Obi-Wan Iwobi,0,James,26
86,339264,524,Neuer Gonna Give You,0,Sarr,26
87,300296,589,Ducks of Bethlehem,0,Minamino,26
88,299717,615,Obi-Wan Iwobi,0,Bergwijn,26


In [78]:
player_info = session.get("https://draft.premierleague.com/api/bootstrap-static")
player_info_json = json.loads(player_info.text)

In [98]:
for i in player_info_json['elements']:
    print(i['elemenid'])

KeyError: 'element'

In [99]:
player_info_json['elements']

'draft_rank': 241,
  'dreamteam_count': 0,
  'ep_next': None,
  'ep_this': None,
  'event_points': 1,
  'first_name': 'Ignacio',
  'form': '1.0',
  'in_dreamteam': False,
  'news': '',
  'news_added': None,
  'news_return': None,
  'news_updated': None,
  'points_per_game': '1.0',
  'second_name': 'Pussetto',
  'squad_number': None,
  'status': 'a',
  'total_points': 4,
  'web_name': 'Pussetto',
  'element_type': 3,
  'team': 18},
 {'id': 605,
  'assists': 0,
  'bonus': 0,
  'bps': 27,
  'clean_sheets': 0,
  'creativity': '0.0',
  'goals_conceded': 5,
  'goals_scored': 0,
  'ict_index': '4.9',
  'influence': '49.6',
  'minutes': 180,
  'own_goals': 0,
  'penalties_missed': 0,
  'penalties_saved': 0,
  'red_cards': 0,
  'saves': 6,
  'threat': '0.0',
  'yellow_cards': 0,
  'added': '2020-01-16T11:07:15.768495Z',
  'chance_of_playing_next_round': None,
  'chance_of_playing_this_round': None,
  'code': 32259,
  'draft_rank': 329,
  'dreamteam_count': 0,
  'ep_next': None,
  'ep_this': Non

In [100]:
player_names = []

for i in range(len(all_players_df)):
    if i % 100 == 0:
        print(len(player_names))
    for el in player_info_json['elements']:
        if all_players_df['player_id'].iloc[i]  == el['id']:
            player_names.append(el['web_name'])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [101]:
all_players_df["player_name"] = player_names

In [96]:
len(player_names)

0

In [104]:
all_players_df.head(20)

,team_id,gameweek,player_id,position,subbed,round_score,player_name
0,333932,1,149,1,N/A,0,Pickford
1,333932,1,204,2,N/A,0,Walker
2,333932,1,122,3,Subbed In,8,Wan-Bissaka
3,333932,1,334,4,N/A,0,Sánchez
4,333932,1,346,5,N/A,1,Lucas Moura
5,333932,1,267,6,N/A,0,Pérez
6,333932,1,448,7,N/A,3,Tielemans
7,333932,1,134,8,N/A,2,Zaha
8,333932,1,113,9,Subbed In,0,Pedro
9,333932,1,339,10,N/A,0,Kane


In [ ]:
# fantastic the "fantasy" and "draft" API's have slightly different player id's. perfect. I haven't found the endpoint to get player scores by week with a draft api so I will need another solution 

In [121]:
draft_player_info = session.get("https://draft.premierleague.com/api/bootstrap-static")
draft_info_json = json.loads(draft_player_info.text)
fantasy_player_info = session.get("https://fantasy.premierleague.com/api/bootstrap-static/")
fantasy_info_json = json.loads(fantasy_player_info.text)

In [122]:
draft_info_json['elements'][133]

{'id': 134,
 'assists': 5,
 'bonus': 4,
 'bps': 262,
 'clean_sheets': 5,
 'creativity': '427.7',
 'goals_conceded': 32,
 'goals_scored': 3,
 'ict_index': '160.6',
 'influence': '439.2',
 'minutes': 2275,
 'own_goals': 0,
 'penalties_missed': 0,
 'penalties_saved': 0,
 'red_cards': 0,
 'saves': 0,
 'threat': '737.0',
 'yellow_cards': 3,
 'added': '2019-06-20T16:47:59.990498Z',
 'chance_of_playing_next_round': 100,
 'chance_of_playing_this_round': 100,
 'code': 82403,
 'draft_rank': 40,
 'dreamteam_count': 0,
 'ep_next': None,
 'ep_this': None,
 'event_points': 5,
 'first_name': 'Wilfried',
 'form': '3.5',
 'in_dreamteam': False,
 'news': '',
 'news_added': '2020-01-06T01:00:17.820409Z',
 'news_return': None,
 'news_updated': None,
 'points_per_game': '3.3',
 'second_name': 'Zaha',
 'squad_number': None,
 'status': 'a',
 'total_points': 87,
 'web_name': 'Zaha',
 'element_type': 3,
 'team': 7}

In [127]:
fantasy_info_json['elements'][140]['web_name']

'Hart'

In [128]:
len(fantasy_info_json['elements'])

623

In [129]:
len(draft_info_json['elements'])

625

In [130]:
draft_web_name = []
draft_id = []
fantasy_id = []
fantasy_web_name = []
for el in draft_info_json['elements']:
    draft_web_name.append(el['web_name'])
    draft_id.append(el['id'])

for el in fantasy_info_json['elements']:
    fantasy_web_name.append(el['web_name'])
    fantasy_id.append(el['id'])


In [131]:
draft_ids_df = pd.DataFrame()
fantasy_ids_df = pd.DataFrame()

draft_ids_df['web_name'] = draft_web_name 
draft_ids_df['id'] = draft_id 
fantasy_ids_df['web_name'] = fantasy_web_name 
fantasy_ids_df['id'] = fantasy_id 

In [138]:
draft_ids_df[draft_ids_df['id'] == 122]

,web_name,id
121,Wan-Bissaka,122


In [137]:
fantasy_ids_df[fantasy_ids_df['id'] == 122]

,web_name,id
327,Wan-Bissaka,122


In [139]:
fantasy_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623 entries, 0 to 622
Data columns (total 2 columns):
web_name    623 non-null object
id          623 non-null int64
dtypes: int64(1), object(1)
memory usage: 9.9+ KB


In [140]:
draft_ids_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625 entries, 0 to 624
Data columns (total 2 columns):
web_name    625 non-null object
id          625 non-null int64
dtypes: int64(1), object(1)
memory usage: 9.9+ KB


In [143]:
difference = []
for i in range(len(draft_ids_df)):
    if draft_ids_df['id'].iloc[i] not in fantasy_id:
        difference.append(draft_ids_df['id'].iloc[i])

In [144]:
difference

[424, 425, 426]

In [149]:
draft_ids_df[draft_ids_df['id'] == 425]

,web_name,id
424,McGoldrick,425


In [170]:
difference = {}
for i in range(len(draft_ids_df)):
    for j in range(len(fantasy_ids_df)):
        if draft_ids_df['web_name'].iloc[i] == fantasy_ids_df['web_name'].iloc[j]:
            difference[draft_ids_df['web_name'].iloc[i]] = [draft_ids_df['id'].iloc[i]]
            difference[draft_ids_df['web_name'].iloc[i]].append(fantasy_ids_df['id'].iloc[j])

         

In [179]:
difference

{'Mustafi': [1, 1],
 'Bellerín': [2, 2],
 'Kolasinac': [3, 3],
 'Maitland-Niles': [4, 4],
 'Sokratis': [5, 5],
 'Monreal': [6, 6],
 'Koscielny': [7, 7],
 'Mavropanos': [8, 8],
 'Jenkinson': [9, 9],
 'Holding': [10, 10],
 'Aubameyang': [11, 11],
 'Lacazette': [12, 12],
 'Nketiah': [13, 13],
 'Leno': [14, 14],
 'Özil': [15, 15],
 'Mkhitaryan': [16, 16],
 'Iwobi': [17, 17],
 'Xhaka': [18, 18],
 'Torreira': [19, 19],
 'Elneny': [20, 20],
 'Guendouzi': [21, 21],
 'El Mohamady': [22, 22],
 'Chester': [23, 23],
 'Taylor': [86, 87],
 'Hause': [25, 25],
 'Kodjia': [26, 26],
 'Nyland': [27, 27],
 'Hourihane': [28, 28],
 'Grealish': [29, 29],
 'El Ghazi': [30, 30],
 'McGinn': [31, 31],
 'Green': [33, 32],
 'Bjarnason': [34, 33],
 'Davis': [35, 34],
 'Lansbury': [36, 35],
 'Duffy': [302, 300],
 'Montoya': [38, 38],
 'Bernardo': [39, 39],
 'Balogun': [40, 40],
 'Bong': [41, 41],
 'Dunk': [42, 42],
 'Murray': [43, 44],
 'Locadia': [44, 45],
 'Andone': [45, 46],
 'Ryan': [46, 47],
 'Button': [47, 48]

In [188]:
for key,value in difference.items():
    if len(value) < 2:
        print(key)

In [166]:
difference['test'][0] == difference['test'][1]

False

In [ ]:

for name in web_name:
    for item in draft_info_json['elements']:
        if web_name == item['web_name']:
            item['id']

In [ ]:
# strictly I should really be keeping a table with all player scores, player information and team info as three separate tables in my database. To make my life easier later though I will combine all this into one table for my db. Hopefully I don't end up regretting this
